In [1]:
import pandas as pd
from os import path
import numpy as np

In [9]:
datasets = ['Adult', 'COMPAS', 'Give_Me_Some_Credit']
ml_methods = ['ANN', 'Linear']

cf_ind = ['ar', 'as', 'cem', 'dice', 'gs', 'wachter']
cf_dep = ['ar-lime', 'clue', 'dice_vae', 'cem-vae', 'face-knn', 'face-eps', 'revise']
cf_methods = cf_ind + cf_dep

measurements = ["ell0", "ell1", "ell2", "ell-inf", "redundancy", "violation", "ynn", "success", "avgtime"]

In [10]:
results = []

for d in datasets:
    for clf in ml_methods:
        for cf in cf_methods:
            filepath = f"../Results/{d}/{clf}/{cf}.csv"

            if path.exists(filepath):
                df_result = pd.read_csv(filepath, index_col=0)
            else:
                df_result = pd.DataFrame([[np.NaN] * len(measurements)], columns=measurements)

            df_result["data"] = d.lower()
            df_result["clf"] = clf.lower()
            df_result["cf_method"] = cf.lower().replace("-", "_")

            results.append(df_result)

df = pd.concat(results)

In [11]:
def approach(method):
    a = "unknown"
    if method in [m.lower().replace("-", "_") for m in cf_ind]:
        a = "independence"
    elif method in [m.lower().replace("-", "_") for m in cf_dep]:
        a = "dependence"
        
    return a

df["approach"] = df["cf_method"].apply(approach)

In [21]:
groupers = ["data", "cf_method", "clf"]

df_agg = df.groupby(groupers, dropna=False, as_index=False).agg(
    count=("ell0", "count"),
    ell0=("ell0", "mean"),
    ell1=("ell1", "mean"),
    ell2=("ell2", "mean"),
    ell_inf=("ell-inf", "mean"),
    redundancy=("redundancy", "mean"),
    violation=("violation", "mean"),
    ynn=("ynn", "first"),
    success=("success", "first"),
    avgtime=("avgtime", "first"),
    approach=("approach", "first")
)

drop_cols = ["ell0", "ell1", "ell2", "ell_inf"]
df_agg = df_agg.drop(drop_cols, axis="columns")

a = df_agg.copy()

### Formatting

In [22]:
df_agg = a

datasets = {
    'give_me_some_credit': "GMC",
    "adult": "Adult",
    "compas": "COMPAS"
}
df_agg["data"] = df_agg["data"].replace(datasets)

cf_methods = {
    "cem_vae": "\\texttt{CEM-VAE}",
    "face_eps": "\\texttt{FACE--EPS}",
    "face_knn": "\\texttt{FACE--KNN}",
    "dice_vae": "\\texttt{EB--CF}",
    "clue": "\\texttt{CLUE}",
    "as": "\\texttt{AS}",
    "ar": "\\texttt{AR(--LIME)}",
    "cem": "\\texttt{CEM}",
    "dice": "\\texttt{DICE}",
    "gs": "\\texttt{GS}",
}
df_agg["cf_method"] = df_agg["cf_method"].replace(cf_methods)
#df_agg["cf_method"] = df_agg["cf_method"].str.upper()

df_agg["count"] = df_agg["count"].astype(int)

cols = {
    "count": "$n$",
    "ynn": "\textit{yNN}",
    "redundancy": "redund.",
    "violation": "violation",
    "success": "success",
    "avgtime": "$\overline{t}(s)$",
}

df_agg = df_agg.rename(cols, axis="columns")


df_dep = df_agg[df_agg["approach"] == "dependence"].drop("approach", axis="columns")
df_ind = df_agg[df_agg["approach"] == "independence"].drop("approach", axis="columns")

In [23]:
def latex_table(df):
        
    df = df.melt(id_vars=["data", "cf_method", "clf"], var_name="measurement")
    
    df = df.pivot_table(index=["data", "cf_method"], columns=["clf", "measurement"])
 
    df = df.T.reindex(cols.values(), level='measurement').T

    df = df.droplevel(0, axis=1)
    df = df.droplevel(0, axis=1)
    df[cols["count"]] = df[cols["count"]].astype(int)
     
    return df

df_latex = latex_table(df_dep)
latex_table(df_ind)

measurement                 $n$  \textit{yNN}   redund.  violation   success  \
data   cf_method                                                               
Adult  \texttt{AR(--LIME)}   33      0.139394  2.727273   1.363636  0.330000   
       \texttt{AS}           25      0.320000  1.720000   0.000000  1.000000   
       \texttt{CEM}         100      0.376000  3.850000   0.000000  1.000000   
       \texttt{DICE}         60      0.803333  1.250000   0.000000  0.600000   
       \texttt{GS}          100      0.390000  3.700000   0.000000  1.000000   
COMPAS \texttt{AR(--LIME)}    1      0.672973  0.000000   1.000000  0.778947   
       \texttt{AS}           11      0.981818  4.272727   0.000000  1.000000   
       \texttt{CEM}         100      0.986000  6.420000   0.000000  1.000000   
       \texttt{DICE}         64      1.000000  4.203125   0.000000  0.640000   
       \texttt{GS}          100      1.000000  9.590000   0.000000  1.000000   
GMC    \texttt{AR(--LIME)}    1      0.983607  5.000000   1.000000  0.871429   
       \texttt{AS}           30      0.566667  0.000000   0.000000  1.000000   
       \texttt{CEM}          70      0.805714  4.957143   1.000000  1.000000   
       \texttt{DICE}        100      0.998000  2.920000   0.020000  1.000000   
       \texttt{GS}           70      0.397143  4.785714   0.000000  1.000000   

measurement                 $\overline{t}(s)$  $n$  \textit{yNN}    redund.  \
data   cf_method                                                              
Adult  \texttt{AR(--LIME)}           3.491508    1      0.264000   3.000000   
       \texttt{AS}                 391.439358   30      0.240000   2.966667   
       \texttt{CEM}                  8.002482  100      0.168000   2.820000   
       \texttt{DICE}                 8.969478  100      0.692000   2.490000   
       \texttt{GS}                   0.011901  100      0.272000   2.030000   
COMPAS \texttt{AR(--LIME)}           1.134961    1      0.000000   0.000000   
       \texttt{AS}                 237.044077    0           NaN        NaN   
       \texttt{CEM}                  5.220907  100      0.498000   6.470000   
       \texttt{DICE}                 6.213149  100      0.950000   4.680000   
       \texttt{GS}                   0.007456  100      0.922000  10.380000   
GMC    \texttt{AR(--LIME)}           2.099734    1      0.956818   8.000000   
       \texttt{AS}                 210.352200   30      0.613333   5.266667   
       \texttt{CEM}                  5.786273  100      0.540000   4.710000   
       \texttt{DICE}                10.230775  100      1.000000   4.870000   
       \texttt{GS}                   0.012016  100      0.550000   6.840000   

measurement                 violation   success  $\overline{t}(s)$  
data   cf_method                                                    
Adult  \texttt{AR(--LIME)}       2.00  0.750000           1.583122  
       \texttt{AS}               0.00  1.000000         248.556749  
       \texttt{CEM}              0.00  1.000000           4.958948  
       \texttt{DICE}             0.00  1.000000           2.638826  
       \texttt{GS}               0.00  1.000000           0.007684  
COMPAS \texttt{AR(--LIME)}       0.00  0.010000           1.829436  
       \texttt{AS}                NaN       NaN                NaN  
       \texttt{CEM}              0.00  1.000000           4.918562  
       \texttt{DICE}             0.00  1.000000           2.420089  
       \texttt{GS}               0.00  1.000000           0.007275  
GMC    \texttt{AR(--LIME)}       1.00  0.907216           0.424235  
       \texttt{AS}               0.00  1.000000         222.557501  
       \texttt{CEM}              0.03  1.000000           5.256922  
       \texttt{DICE}             0.00  1.000000           5.785889  
       \texttt{GS}               0.00  1.000000           0.007357

In [24]:
df_latex = latex_table(df_ind)
latex = df_latex.to_latex(
        float_format="{:0.2f}".format, 
        na_rep="--",
        #sparsify=False,
        multirow=True,
        multicolumn=True,
        escape=False,
#         bold_rows=True,
        label="method_comparison_dependend",
        caption="Dependent methods results"
    )

print(latex)

\begin{table}
\centering
\caption{Dependent methods results}
\label{method_comparison_dependend}
\begin{tabular}{llrrrrrrrrrrrr}
\toprule
    & measurement &  $n$ &  \textit{yNN} &  redund. &  violation &  success &  $\overline{t}(s)$ &  $n$ &  \textit{yNN} &  redund. &  violation &  success &  $\overline{t}(s)$ \\
data & cf_method &      &               &          &            &          &                    &      &               &          &            &          &                    \\
\midrule
\multirow{5}{*}{Adult} & \texttt{AR(--LIME)} &   33 &          0.14 &     2.73 &       1.36 &     0.33 &               3.49 &    1 &          0.26 &     3.00 &       2.00 &     0.75 &               1.58 \\
    & \texttt{AS} &   25 &          0.32 &     1.72 &       0.00 &     1.00 &             391.44 &   30 &          0.24 &     2.97 &       0.00 &     1.00 &             248.56 \\
    & \texttt{CEM} &  100 &          0.38 &     3.85 &       0.00 &     1.00 &               8.00 &  100 &     